# Example 6.9

In [1]:
import numpy as np
from IPython.display import display, Math

In [2]:
p1 = np.array([9, 6, 9])
p2 = np.array([0, 2])
p3 = np.array([3, 1, 2])

In [3]:
g1 = np.hstack([np.array([[9], [3]]),
                np.array([[6], [2]]),
                np.array([[9], [7]])])

g2 = np.hstack([np.array([[0], [4]]),
                np.array([[2], [0]])])

g3 = np.hstack([np.array([[3], [8]]),
                np.array([[1], [9]]),
                np.array([[2], [7]])])

In [4]:
# Rearrange the groups to 3D (i,j,k). The depth (i) is the group. The height (j) is the measurement. The width (k) is the observation.
x = np.stack((g1, np.hstack((g2, np.array([[np.nan], [np.nan]]))), g3))
x

array([[[ 9.,  6.,  9.],
        [ 3.,  2.,  7.]],

       [[ 0.,  2., nan],
        [ 4.,  0., nan]],

       [[ 3.,  1.,  2.],
        [ 8.,  9.,  7.]]])

In [5]:
# Mean for each group.
xbarg = np.stack([np.nanmean(x[i,:,:], axis=1)[:, np.newaxis] for i in range(x.shape[0])])
# Global mean.
xbar = np.vstack([np.nanmean(x[:,i,:].flatten()) for i in range(x.shape[1])])

In [6]:
for i in range(2):
    mean_matrix = xbar[i] * np.ones((3,3))
    tr_matrix = (xbarg[:,i,:] - xbar[i]) * np.ones((3,3))
    res_matrix = (x[:,i,:] - xbarg[:,i,:] * np.ones((3,3)))
    display(Math(fr'\text{{Observation }}{i+1}:'))
    display(Math(fr'''\left[
                      \begin{{array}}{{rrr}}
                            {x[0,i,0]:.0f} & {x[0,i,1]:.0f} & {x[0,i,2]:.0f} \\
                            {x[1,i,0]:.0f} & {x[1,i,1]:.0f} &            \\
                            {x[2,i,0]:.0f} & {x[2,i,1]:.0f} & {x[2,i,2]:.0f}
                      \end{{array}}
                      \right] = '''
                fr'''\left[
                      \begin{{array}}{{rrr}}
                            {mean_matrix[0,0]:.0f} & {mean_matrix[0,1]:.0f} & {mean_matrix[0,2]:.0f} \\
                            {mean_matrix[1,0]:.0f} & {mean_matrix[1,1]:.0f} &                \\
                            {mean_matrix[2,0]:.0f} & {mean_matrix[2,1]:.0f} & {mean_matrix[2,2]:.0f}
                      \end{{array}}
                      \right] + '''
                fr'''\left[
                      \begin{{array}}{{rrr}}
                            {tr_matrix[0,0]:.0f} & {tr_matrix[0,1]:.0f} & {tr_matrix[0,2]:.0f} \\
                            {tr_matrix[1,0]:.0f} & {tr_matrix[1,1]:.0f} &              \\
                            {tr_matrix[2,0]:.0f} & {tr_matrix[2,1]:.0f} & {tr_matrix[2,2]:.0f}
                      \end{{array}}
                      \right] + '''
                fr'''\left[
                      \begin{{array}}{{rrr}}
                            {res_matrix[0,0]:.0f} & {res_matrix[0,1]:.0f} & {res_matrix[0,2]:.0f} \\
                            {res_matrix[1,0]:.0f} & {res_matrix[1,1]:.0f} &               \\
                            {res_matrix[2,0]:.0f} & {res_matrix[2,1]:.0f} & {res_matrix[2,2]:.0f}
                      \end{{array}}
                      \right]'''

                      ))
    display(Math(r'\hspace{0.0cm}\text{(observation)}\hspace{0.9cm}\text{(mean)}\hspace{1.0cm}\text{(treatment effect)}\hspace{1.1cm}\text{(residual)}'))
    SS_obs = (x[:,i,:][~np.isnan(x[:,i,:])]**2).sum()
    SS_mean = (mean_matrix[~np.isnan(x[:,i,:])]**2).sum()
    SS_tr = (tr_matrix[~np.isnan(x[:,i,:])]**2).sum()
    SS_res = (res_matrix[~np.isnan(x[:,i,:])]**2).sum()
    SS_tot = ((x[:,i,:] - xbar[i])[~np.isnan(x[:,i,:])]**2).sum()
    display(Math(r'\text{and}'))
    display(Math(r'\text{SS}_{\text{obs}} = \text{SS}_{\text{mean}} + \text{SS}_{\text{tr}} + \text{SS}_{\text{res}}'))
    display(Math(fr'{SS_obs:.0f} = {SS_mean:.0f} + {SS_tr:.0f} + {SS_res:.0f}'))
    display(Math(fr'\text{{Total SS (corrected)}} = \text{{SS}}_{{\text{{obs}}}} - \text{{SS}}_{{\text{{mean}}}} = {SS_obs:.0f} - {SS_mean:.0f} = {SS_tot:.0f}'))
    print()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>